In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv
import urllib
from selenium.webdriver.common.action_chains import ActionChains
import requests
import json
import re
import tqdm
import urllib.request
import os
from tqdm import tqdm
from selenium import webdriver
from selenium.common.exceptions import TimeoutException,StaleElementReferenceException,InvalidArgumentException,ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pickle5 as pickle
list_of_hotel_link=pickle.load(open("list_of_hotel_link.pickle","rb"))   

In [5]:
list_of_hotel_link.index("https://www.agoda.com/vi-vn/villa-panda-sanctuary-ho-tram-resort/hotel/all/vung-tau-vn.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2021-05-31&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=VND&isFreeOccSearch=false&los=1&searchrequestid=bc8a11fa-8c2f-41a8-b5b7-cf1f17022781")

11183

In [ ]:

# link_to_download=["https://www.agoda.com/vi-vn/search?city=15932"]

# # driver = webdriver.Chrome()
# for url in link_to_download:
#     driver.get(url)
#     #Scroll Down
#     for _ in tqdm(range(70)):
#             for _ in range(30):
#                 try: 
#                     html = driver.find_element_by_tag_name('html')
#                     html.send_keys(Keys.END)
#                 except:
#                     continue
#             sleep(2)
#             #Get_All_Hotels_Link
#             for element in driver.find_elements_by_class_name("PropertyCard__Link"):
#                 try:
#                     list_of_hotel_link.append(element.get_attribute("href"))
#                 except:
#                     continue
#             flag=True
#             i=0
#             while flag:
              
#                 try:
#                     html = driver.find_element_by_id('paginationNext')
#                     html.click()
#                     flag=False
#                 except:
#                     i+=1
#                     if i==50:
#                         flag=False
#                     continue
#            # pickle.dump(list_of_hotel_link,open("up_grade_list_of_hotel_link.pickle","wb"))            


In [6]:
driver = webdriver.Chrome()

user_comments= pickle.load(open("user_comments.pickle","rb"))

len(user_comments)

97923

In [7]:
from selenium.common.exceptions import ElementClickInterceptedException
#user_comments=[]
for url in tqdm(list_of_hotel_link[11184:],total=len(list_of_hotel_link[11184:])):
    ##Choosing comments in Vnese only
    try:
        driver.get(url)
    except InvalidArgumentException:
        continue 
        
    # timeout = 5
    # try:
    #     element_present = EC.presence_of_eslement_located((By.XPATH, '//*[@id="reviewFilterSection"]/div[1]/div[3]'))
    #     WebDriverWait(driver, timeout).until(element_present)
    # except TimeoutException:
    #     print("Timed out waiting for page to load")
    # html = driver.find_element_by_xpath('//*[@id="reviewFilterSection"]/div[1]/div[3]')
    # html.click()
    try:
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'Review__FilterWrapper')))
        html=driver.find_element_by_class_name("Review__FilterWrapper").find_element_by_class_name("Review__FilterContainer")
    except TimeoutException:
        continue
    try:
        WebDriverWait(html, 20).until(EC.element_to_be_clickable((By.TAG_NAME, 'div')))
        for elements in html.find_elements_by_tag_name("div"):
            if elements.get_attribute("data-selenium")=="reviews-language-filter":
                html=elements
                sleep(5)
                elements.click()

                # try:
                #      WebDriverWait(html, 5).until(EC.element_to_be_clickable((By.TAG_NAME, 'div'))).click()
                # except TimeoutException:
                #     continue
    except (TimeoutException,ElementClickInterceptedException):
        continue

    for elements in html.find_elements_by_tag_name("li"):
        if elements.find_elements_by_tag_name("span")[2].text=="Tiếng Việt":
            try:
                WebDriverWait(elements, 20).until(EC.element_to_be_clickable((By.TAG_NAME, 'span'))).click()
            except TimeoutException:
                continue
    #get_All_comments
    for elements in driver.find_elements_by_class_name("Review-comment-bodyText"):
        user_comments.append(elements.text)
    #Next page
    sleep(3)
    for _ in range(15):
        try:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviewSection"]/div[4]/div/span[3]'))).click()
        except (ElementClickInterceptedException,TimeoutException,StaleElementReferenceException):
            break
        for elements in driver.find_elements_by_class_name("Review-comment-bodyText"):
            try:
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'Review-comment-bodyText')))
                user_comments.append(elements.text)
            except (TimeoutException,StaleElementReferenceException):
                print("Xu Ca Na")
                continue
    pickle.dump(user_comments,open("user_comments.pickle","wb"))


  0%|          | 24/20971 [08:40<132:12:23, 22.72s/it]

ElementClickInterceptedException: Message: element click intercepted: Element <div class="select-dropdown-wrapper Review__FilterContainer__Dropbox" data-selenium="reviews-language-filter" value="0" data-element-name="reviews-language-filter">...</div> is not clickable at point (724, 747). Other element would receive the click: <p class="BackToSearch-toRoomGridText">...</p>
  (Session info: chrome=90.0.4430.212)
